## ExMAS
> Showcase for mutually exclusive  rides in the solution

In [1]:
import os
import math
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
cwd = os.getcwd()
%load_ext autoreload
%autoreload 2

In [2]:
from IPython.display import display
pd.options.display.max_columns = None

In [3]:
os.chdir(os.path.join(cwd,'../../..'))
import ExMAS.main
import ExMAS.utils
from ExMAS.utils import inData as inData
from ExMAS.main import matching

In [4]:
params = ExMAS.utils.get_config('ExMAS/spinoffs/game/game_toy.json') # load the default 
params.t0 = pd.to_datetime(params.t0)
params.matching_obj = 'u_pax'
inData = ExMAS.utils.load_G(inData, params, stats=True)  # download the graph
inData.requests = ExMAS.utils.load_requests('ExMAS/spinoffs/game/toy.csv')

In [5]:
from ExMAS.main import init_log
params.logger_level = 'INFO'
inData.logger = init_log(params)

In [6]:
params.shared_discount = 0.5
inData = ExMAS.main(inData, params, plot = False)
KPIs = inData.sblts.res.to_frame('u_pax')

09-12-20 10:12:34-INFO-Initializing pairwise trip shareability between 10 and 10 trips.
09-12-20 10:12:34-INFO-creating combinations
09-12-20 10:12:34-INFO-90	 nR*(nR-1)
09-12-20 10:12:34-INFO-Reduction of feasible pairs by 90.00%
09-12-20 10:12:34-INFO-Degree 2 	Completed
09-12-20 10:12:34-INFO-trips to extend at degree 2 : 21
09-12-20 10:12:34-INFO-At degree 2 feasible extensions found out of 4 searched
09-12-20 10:12:34-INFO-Degree 3 	Completed
09-12-20 10:12:34-INFO-trips to extend at degree 3 : 4
09-12-20 10:12:34-INFO-At degree 3 feasible extensions found out of 0 searched
09-12-20 10:12:34-INFO-Degree 4 	Completed
09-12-20 10:12:34-INFO-No more trips to exted at degree 4
09-12-20 10:12:34-INFO-Matching 10 trips to 35 rides in order to min u_pax
09-12-20 10:12:34-INFO-Problem solution: Optimal. 
Total costs for single trips:             32 
reduced by matching to:                   26
09-12-20 10:12:34-INFO-Calculations  completed
09-12-20 10:12:34-INFO-DotMap(VehHourTrav=1830, V

### inspect the solution

In [7]:
rides = inData.sblts.rides
rides[rides.selected==1][['indexes']]

,indexes
3,[3]
8,[8]
11,"[0, 7]"
17,"[5, 6]"
20,"[9, 1]"
26,"[2, 4]"


let's exclude pair `[3,8]`

In [8]:
inData.sblts.mutually_exclusives = [[3,8]]

inData = matching(inData, params)
rides = inData.sblts.rides
rides[rides.selected==1][['indexes']]

09-12-20 10:12:35-INFO-Matching 10 trips to 35 rides in order to min u_pax
09-12-20 10:12:35-INFO-Adding 1 mutually exlcusive constrains
09-12-20 10:12:35-INFO-Problem solution: Optimal. 
Total costs for single trips:             32 
reduced by matching to:                   26


,indexes
3,[3]
7,[7]
10,"[0, 5]"
20,"[9, 1]"
26,"[2, 4]"
30,"[6, 8]"


now it is gone from the solution, (3 remained, but 8 is gone)

so now let's remove also `[30,26]`

In [9]:
inData.sblts.mutually_exclusives = [[3,8],[26,30]]

inData = matching(inData, params)
rides = inData.sblts.rides
rides[rides.selected==1][['indexes']]

09-12-20 10:12:35-INFO-Matching 10 trips to 35 rides in order to min u_pax
09-12-20 10:12:35-INFO-Adding 2 mutually exlcusive constrains
09-12-20 10:12:35-INFO-Problem solution: Optimal. 
Total costs for single trips:             32 
reduced by matching to:                   27


,indexes
3,[3]
7,[7]
10,"[0, 5]"
15,"[4, 2]"
20,"[9, 1]"
30,"[6, 8]"


revert

In [10]:
inData.sblts.mutually_exclusives = []

inData = matching(inData, params)
rides = inData.sblts.rides
rides[rides.selected==1][['indexes']]

09-12-20 10:12:35-INFO-Matching 10 trips to 35 rides in order to min u_pax
09-12-20 10:12:35-INFO-Problem solution: Optimal. 
Total costs for single trips:             32 
reduced by matching to:                   26


,indexes
3,[3]
8,[8]
11,"[0, 7]"
17,"[5, 6]"
20,"[9, 1]"
26,"[2, 4]"


---
(c) Rafał Kucharski, Delft, 2020